In [7]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import scipy.io as sio
import cmaps
from matplotlib.colors import LinearSegmentedColormap

In [2]:
import roms_workspace as rw
import scipy as sci
import os

In [3]:
my_rootnl = '/Volumes/Elements SE/roms4dvar_ecs'
nl_workspace_info = {
                     'freerun_detide_v3.7/outputs':'ROMS 3.7',
                     'freerun_detide_v4.1/outputs':'ROMS 4.1 NO SCORRECTION',
                        'freerun_detide_v4.1_SC/outputs':'ROMS 4.1 SCORRECTION',
                    # 'run17_v4/outputs':'FORWARD drggrd',
}
var = 'salt'
areas = {
        #  'bh': ('Bohai Sea', [0,7]),
        # 'ys': ('Yellow Sea', [6.5,11]),
        # 'cre': ('Changjiang River Estuary',[10.5,14.5]),
        # 'ks': ('Kuroshio Current System', [21.5,24]),
        'all': ('East China Sea', [10,25])
}
nl_Workspaces = []
for i in range(len(nl_workspace_info)):
    nl_Workspaces.append(rw.Nl_workspace( MY_ROOT         =my_rootnl,
                                    workspace_name  =list(nl_workspace_info.keys())[i],
                                    workspace_detail=list(nl_workspace_info.values())[i]
                                   ))
    nl_Workspaces[i].load_mtfroms(isAna=False,Surf=True)

Workspace: /Volumes/Elements SE/roms4dvar_ecs/freerun_detide_v3.7/outputs
Workspace: /Volumes/Elements SE/roms4dvar_ecs/freerun_detide_v4.1/outputs
Workspace: /Volumes/Elements SE/roms4dvar_ecs/freerun_detide_v4.1_SC/outputs


In [6]:
var = 'salt'

In [8]:
nl_Vars = {}
for iw in range(len((nl_workspace_info))):
    for area in list(areas.keys()):
        print('loading %s in %s  area...'%(var,area))
        if iw == 0:
            nl_Vars[area] = []
        # loading data from FORWARD model outputs
        nl_Vars[area].append(nl_Workspaces[iw].pull_var2d(varname=         var, area=area,time_range=('2004-07-25','2004-08-01')))

loading salt in all  area...
loading salt in all  area...
loading salt in all  area...


In [28]:
len(nl_Workspaces)

2

In [9]:
def _forward(x):
    return x**3


def _inverse(x):
    return np.power(x,1/3)
norm = colors.FuncNorm((_forward, _inverse), vmin=5, vmax=35)

In [ ]:
lon_bdy = [120,133.5];
lat_bdy = [20,31];
vmin_sst = 22
vmax_sst = 26
my_sst_color = sio.loadmat('LYG_rainbow.mat')['rainbow']
my_sst = LinearSegmentedColormap.from_list('sst',my_sst_color, N = 256)
my_div_color = np.array(  [
                 [0,0,123],
                [9,32,154],
                [22,58,179],
                [34,84,204],
                [47,109,230],
                [63,135,247],
                [95,160,248],
                [137,186,249],
                [182,213,251],
                [228,240,254],
                [255,255,255],
                [250,224,224],
                [242,164,162],
                [237,117,113],
                [235,76,67],
                [233,52,37],
                [212,45,31],
                [188,39,26],
                [164,33,21],
                [140,26,17],
                [117,20,12]
                ])/255
my_div = LinearSegmentedColormap.from_list('div',my_div_color, N = 256)


d_interval = 2

# iwda = 0
plt.rcParams['font.family'] = 'Lucida Grande'
area = 'all'
# for area in ['bh','ys','ks',]:
figsize = (11,5)
lon_bdy = [116,136];
lat_bdy = [21,38.5];
lon_interval = 5
lat_interval = 5
vmin_sst = 5
vmax_sst = 35
norm = colors.FuncNorm((_forward, _inverse), vmin=vmin_sst, vmax=vmax_sst)
vmin_diff = -0.5
vmax_diff = 0.5
d_interval = 5
bc = 0.93
for iwork in range(len(nl_Workspaces)):
    for it in range(len(nl_Vars[area][iwork].ocean_time)):
    # for it in range(1):
        print(iwork,it)
        fig = plt.figure(figsize=figsize, constrained_layout = True)
        #===============================observation spatial pattern
        ax = fig.add_subplot(projection = ccrs.Mercator())
        for bnd in ax.spines.keys():
            ax.spines[bnd].set_linewidth('1.5')
        ax.add_feature(cfeature.COASTLINE.with_scale('10m'), color = 'gray',zorder = 2,linewidth =1,)
        ax.add_feature(cfeature.LAND.with_scale('10m'),facecolor='lightgray',zorder = 1,)
        ax.set_extent((lon_bdy[0],lon_bdy[-1],lat_bdy[0],lat_bdy[-1]),crs = ccrs.PlateCarree())
        ax.set_xticks(np.arange(lon_bdy[0],lon_bdy[-1]+lon_interval,lon_interval),crs=ccrs.PlateCarree())
        ax.set_yticks(np.arange(lat_bdy[0],lat_bdy[-1]+lat_interval,lat_interval),crs=ccrs.PlateCarree())
        lon_formatter = LongitudeFormatter(number_format='.0f',
                                        degree_symbol='°',
                                        dateline_direction_label=True)
        lat_formatter = LatitudeFormatter(number_format='.0f',
                                            degree_symbol='°')
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        ax.tick_params(direction = 'out', length = 3, width = 1.5, labelsize = 12)
        c = ax.pcolormesh(nl_Vars[area][iwork].lon,nl_Vars[area][iwork].lat,
                  nl_Vars[area][iwork].value[it,:,:],
                          cmap = cmaps.NCV_bright,
                  # cmap = my_sst, 
                  norm = norm,
                  # cmap = my_sst,vmin = vmin_sst,vmax=vmax_sst,

               transform = ccrs.PlateCarree(),zorder = -1)
        ax.set_title('SSS '+nl_Workspaces[iwork].workspace_detail,fontsize=16)
        ax.text(lon_bdy[-1]*bc,lat_bdy[-1]*0.99,str(nl_Vars[area][iwork].ocean_time.data[it])[:10]+'-%sH'%(str(nl_Vars[area][iwork].ocean_time.data[it])[11:13]),fontsize = 12,transform = ccrs.PlateCarree())
        position=fig.add_axes([0.335,-0.05, 0.325, 0.045,])
        cb = fig.colorbar(c, cax = position, orientation='horizontal',ticks = np.arange(vmin_sst,vmax_sst+d_interval,d_interval))
        for bnd in position.spines.keys():
                position.spines[bnd].set_linewidth('1.5')
        position.tick_params(direction = 'out',  length = 3, width = 1.5, labelsize = 10)

        cb.set_label('PSU',fontsize = 12) 
        plt.savefig(nl_Workspaces[iwork].workspace_subdir+
                    '/fig_evolution/2004sss_%s.jpeg'%(str(it)),dpi=300,bbox_inches='tight')
        plt.close()

0 0


/Users/onecountrylee/opt/anaconda3/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1797: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  result = matplotlib.axes.Axes.pcolormesh(self, *args, **kwargs)
/Users/onecountrylee/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:825: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/Users/onecountrylee/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:877: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Us

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 